# Predicting Podcast Time
**Author: Benson Wainaina**

In this notebook, I aim to predict the amount of time someone listens to a podcast. I'll be exploring various techniques such as linear regression, knearest neighbors, support vector machine, decision trees, random rainforest, neural networks, and ensemble methods. 